# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 2:44PM,242472,19,ADV80004386,"AdvaGen Pharma, Ltd",Released
1,Jul 21 2022 2:44PM,242472,19,ADV80004387,"AdvaGen Pharma, Ltd",Released
2,Jul 21 2022 2:44PM,242472,19,ADV80004388,"AdvaGen Pharma, Ltd",Released
3,Jul 21 2022 2:44PM,242472,19,ADV80004389,"AdvaGen Pharma, Ltd",Released
4,Jul 21 2022 2:44PM,242472,19,ADV80004390,"AdvaGen Pharma, Ltd",Released
5,Jul 21 2022 2:44PM,242472,19,ADV80004391,"AdvaGen Pharma, Ltd",Released
6,Jul 21 2022 2:43PM,242471,19,DEX0007234,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
7,Jul 21 2022 2:43PM,242471,19,DEX0007235,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
8,Jul 21 2022 2:41PM,242470,10,0085899481,ISDIN Corporation,Released
9,Jul 21 2022 2:39PM,242469,16,8301536,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,242468,Released,1
37,242469,Released,2
38,242470,Released,1
39,242471,Released,2
40,242472,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242468,NaN,1.0
242469,NaN,2.0
242470,NaN,1.0
242471,NaN,2.0
242472,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242348,52.0,26.0
242375,0.0,1.0
242376,0.0,1.0
242390,0.0,1.0
242391,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242348,52,26
242375,0,1
242376,0,1
242390,0,1
242391,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242348,52,26
1,242375,0,1
2,242376,0,1
3,242390,0,1
4,242391,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242348,52,26
1,242375,,1
2,242376,,1
3,242390,,1
4,242391,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 2:44PM,242472,19,"AdvaGen Pharma, Ltd"
6,Jul 21 2022 2:43PM,242471,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
8,Jul 21 2022 2:41PM,242470,10,ISDIN Corporation
9,Jul 21 2022 2:39PM,242469,16,Sartorius Bioprocess Solutions
11,Jul 21 2022 2:17PM,242468,10,"Citieffe, Inc."
12,Jul 21 2022 2:06PM,242450,20,"ACI Healthcare USA, Inc."
18,Jul 21 2022 1:47PM,242467,10,Eye Pharma Inc
19,Jul 21 2022 1:44PM,242466,10,"Methapharm, Inc."
22,Jul 21 2022 1:44PM,242465,10,"Methapharm, Inc."
25,Jul 21 2022 1:40PM,242464,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 21 2022 2:44PM,242472,19,"AdvaGen Pharma, Ltd",,6
1,Jul 21 2022 2:43PM,242471,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,2
2,Jul 21 2022 2:41PM,242470,10,ISDIN Corporation,,1
3,Jul 21 2022 2:39PM,242469,16,Sartorius Bioprocess Solutions,,2
4,Jul 21 2022 2:17PM,242468,10,"Citieffe, Inc.",,1
5,Jul 21 2022 2:06PM,242450,20,"ACI Healthcare USA, Inc.",,6
6,Jul 21 2022 1:47PM,242467,10,Eye Pharma Inc,,1
7,Jul 21 2022 1:44PM,242466,10,"Methapharm, Inc.",,3
8,Jul 21 2022 1:44PM,242465,10,"Methapharm, Inc.",,3
9,Jul 21 2022 1:40PM,242464,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 2:44PM,242472,19,"AdvaGen Pharma, Ltd",6,
1,Jul 21 2022 2:43PM,242471,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,
2,Jul 21 2022 2:41PM,242470,10,ISDIN Corporation,1,
3,Jul 21 2022 2:39PM,242469,16,Sartorius Bioprocess Solutions,2,
4,Jul 21 2022 2:17PM,242468,10,"Citieffe, Inc.",1,
5,Jul 21 2022 2:06PM,242450,20,"ACI Healthcare USA, Inc.",6,
6,Jul 21 2022 1:47PM,242467,10,Eye Pharma Inc,1,
7,Jul 21 2022 1:44PM,242466,10,"Methapharm, Inc.",3,
8,Jul 21 2022 1:44PM,242465,10,"Methapharm, Inc.",3,
9,Jul 21 2022 1:40PM,242464,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 2:44PM,242472,19,"AdvaGen Pharma, Ltd",6,
1,Jul 21 2022 2:43PM,242471,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,
2,Jul 21 2022 2:41PM,242470,10,ISDIN Corporation,1,
3,Jul 21 2022 2:39PM,242469,16,Sartorius Bioprocess Solutions,2,
4,Jul 21 2022 2:17PM,242468,10,"Citieffe, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 2:44PM,242472,19,"AdvaGen Pharma, Ltd",6.0,NaN
1,Jul 21 2022 2:43PM,242471,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,NaN
2,Jul 21 2022 2:41PM,242470,10,ISDIN Corporation,1.0,NaN
3,Jul 21 2022 2:39PM,242469,16,Sartorius Bioprocess Solutions,2.0,NaN
4,Jul 21 2022 2:17PM,242468,10,"Citieffe, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 2:44PM,242472,19,"AdvaGen Pharma, Ltd",6.0,0.0
1,Jul 21 2022 2:43PM,242471,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,0.0
2,Jul 21 2022 2:41PM,242470,10,ISDIN Corporation,1.0,0.0
3,Jul 21 2022 2:39PM,242469,16,Sartorius Bioprocess Solutions,2.0,0.0
4,Jul 21 2022 2:17PM,242468,10,"Citieffe, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4364047,57.0,2.0
12,484891,1.0,1.0
16,1454501,10.0,0.0
19,969821,10.0,0.0
20,727237,45.0,52.0
21,1212073,5.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4364047,57.0,2.0
1,12,484891,1.0,1.0
2,16,1454501,10.0,0.0
3,19,969821,10.0,0.0
4,20,727237,45.0,52.0
5,21,1212073,5.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,57.0,2.0
1,12,1.0,1.0
2,16,10.0,0.0
3,19,10.0,0.0
4,20,45.0,52.0
5,21,5.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,12,Released,1.0
2,16,Released,10.0
3,19,Released,10.0
4,20,Released,45.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,2.0,1.0,0.0,0.0,52.0,0.0
Released,57.0,1.0,10.0,10.0,45.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,2.0,1.0,0.0,0.0,52.0,0.0
1,Released,57.0,1.0,10.0,10.0,45.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,2.0,1.0,0.0,0.0,52.0,0.0
1,Released,57.0,1.0,10.0,10.0,45.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()